In [1]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

from ba_utils.data_generator import generate_split_merge_data
#from ba_utils.visualization import visualize_graphs, draw_rug_from_graphs

import ba_utils.visualization as visualization
import ba_utils.network_rugs as nrugs

import networkx as nx

from ba_utils.orderings import get_priority_bfs_ordering, get_DFS_ordering, get_BFS_ordering

# Global data storage
graphs_data = {}

def start_gui():
    global graphs_data
    root = tk.Tk()
    root.title("NetworkRug Generator & Visualizer")
    root.geometry("1800x900")

    left_frame = ttk.Frame(root, width=800)
    right_frame = ttk.Frame(root, width=1000)
    left_frame.pack(side='left', fill='both', expand=True)
    right_frame.pack(side='right', fill='both', expand=True)

    # Input Variables
    num_nodes_var = tk.IntVar(value=30)
    num_steps_var = tk.IntVar(value=10)
    num_groups_var = tk.IntVar(value=1)
    split_time_var = tk.IntVar(value=2)
    split_duration_var = tk.IntVar(value=4)
    merge_time_var = tk.IntVar(value=0)
    merge_duration_var = tk.IntVar(value=0)

    def labeled_entry(parent, text, variable, row):
        ttk.Label(parent, text=text).grid(row=row, column=0, sticky='e', padx=5)
        ttk.Entry(parent, textvariable=variable).grid(row=row, column=1, padx=5)

    for i, (label, var) in enumerate([
        ("Num Nodes", num_nodes_var),
        ("Num Steps", num_steps_var),
        ("Initial Groups", num_groups_var),
        ("Split Time", split_time_var),
        ("Split Duration", split_duration_var),
        ("Merge Time", merge_time_var),
        ("Merge Duration", merge_duration_var)
    ]):
        labeled_entry(left_frame, label, var, i)

    # Canvas placeholder
    left_canvas = ttk.Frame(left_frame)
    left_canvas.grid(row=9, column=0, columnspan=2, pady=10, sticky='nsew')

    def generate_and_visualize():
        global graphs_data
        graphs, _, _ = generate_split_merge_data(
            num_nodes=num_nodes_var.get(),
            num_steps=num_steps_var.get(),
            initial_groups=num_groups_var.get(),
            split_time=split_time_var.get(),
            split_duration=split_duration_var.get(),
            merge_time=merge_time_var.get(),
            merge_duration=merge_duration_var.get()
        )
        graphs_data = graphs
        # --- Embed matplotlib plot ---
        for widget in left_canvas.winfo_children():
            widget.destroy()

        fig = plt.Figure(figsize=(8, 5))
        timestamps = list(graphs.keys())
        cols = 5
        rows = (len(timestamps) + cols - 1) // cols

        for i, (timestamp, G) in enumerate(graphs.items()):
            ax = fig.add_subplot(rows, cols, i + 1)
            pos = nx.spring_layout(G)
            nx.draw(G, pos, with_labels=True, node_size=300, font_size=8, font_color='white', ax=ax)
            ax.set_title(str(timestamp), fontsize=8)

        canvas = FigureCanvasTkAgg(fig, master=left_canvas)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
        print("Graphs visualized")

    ttk.Button(left_frame, text="Generate & Show Graphs", command=generate_and_visualize).grid(row=8, column=0, columnspan=2, pady=10)

    # Right Controls
    color_options = ['id', 'id2', 'id3', 'degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'eigenvector_centrality']
    ordering_options = ['priority', 'dfs', 'bfs']
    
    color_var = tk.StringVar(value='closeness_centrality')
    order_var = tk.StringVar(value='priority')
    pixel_var = tk.IntVar(value=6)
    label_var = tk.BooleanVar()

    ttk.Label(right_frame, text="Color Encoding").pack()
    ttk.Combobox(right_frame, textvariable=color_var, values=color_options).pack()
    
    ttk.Label(right_frame, text="Ordering").pack()
    ttk.Combobox(right_frame, textvariable=order_var, values=ordering_options).pack()

    ttk.Checkbutton(right_frame, text="Show Labels", variable=label_var).pack()
    ttk.Label(right_frame, text="Pixel Size").pack()
    ttk.Scale(right_frame, from_=2, to=20, variable=pixel_var, orient='horizontal').pack()

    rug_canvas = ttk.Frame(right_frame)
    rug_canvas.pack(fill='both', expand=True)

    def render_rug():
        if not graphs_data:
            return
        fig = plt.Figure()
        ax = fig.add_subplot(111)
        
        ordering_fn = {
            'priority': get_priority_bfs_ordering,
            'dfs': get_DFS_ordering,
            'bfs': lambda g: get_BFS_ordering(g, sorting_key='weight')
        }[order_var.get()]

        ordering = ordering_fn(graphs_data)
        '''visualization.draw_rug_from_graphs(
            graphs_data,
            ordering=ordering,
            color_encoding=color_var.get(),
            labels=label_var.get(),
            pixel_size=pixel_var.get(),
            ax=ax
        )'''
        
        nrugs.draw_networkrug(graphs_data,
            color_encoding=color_var.get(),
            labels=label_var.get(),
            pixel_size=pixel_var.get(),
            ax=ax)
        
        for widget in rug_canvas.winfo_children():
            widget.destroy()

        canvas = FigureCanvasTkAgg(fig, master=rug_canvas)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)

    ttk.Button(right_frame, text="Render Rug", command=render_rug).pack(pady=10)
    root.mainloop()



In [3]:
# Call this to launch GUI
start_gui()

In [ ]:
# Left Side generate Graph, right side generate Rug


import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import networkx as nx

from ba_utils.data_generator import generate_dynamic_graphs, generate_proportional_transition
import ba_utils.network_rugs as nrugs
from ba_utils.orderings import get_priority_bfs_ordering, get_DFS_ordering, get_BFS_ordering

graphs_data = {}

def start_gui():
    global graphs_data
    root = tk.Tk()
    root.title("Enhanced NetworkRug GUI")
    root.geometry("1800x900")

    # --- Frames ---
    left_frame = ttk.Frame(root, width=800)
    right_frame = ttk.Frame(root, width=1000)
    left_frame.pack(side='left', fill='both', expand=True)
    right_frame.pack(side='right', fill='both', expand=True)

    # --- Base Controls ---
    num_nodes_var = tk.IntVar(value=50)
    num_steps_var = tk.IntVar(value=100)
    intra_strength_var = tk.DoubleVar(value=0.8)
    inter_strength_var = tk.DoubleVar(value=0.1)
    mode_var = tk.StringVar(value='timed')

    labeled = lambda text, var, r: (ttk.Label(left_frame, text=text).grid(row=r, column=0, sticky='e'),
                                    ttk.Entry(left_frame, textvariable=var).grid(row=r, column=1))

    labeled("Num Nodes", num_nodes_var, 0)
    labeled("Timesteps", num_steps_var, 1)
    labeled("Intra Strength", intra_strength_var, 2)
    labeled("Inter Strength", inter_strength_var, 3)

    # --- Mode Selection ---
    ttk.Label(left_frame, text="Generation Mode").grid(row=4, column=0)
    mode_dropdown = ttk.Combobox(left_frame, textvariable=mode_var, values=["timed", "proportioned"])
    mode_dropdown.grid(row=4, column=1)

    dynamic_widgets_frame = ttk.Frame(left_frame)
    dynamic_widgets_frame.grid(row=5, column=0, columnspan=2, pady=5)

    input_state_var = tk.StringVar()
    final_state_var = tk.StringVar()
    split_events_var = tk.StringVar()
    merge_events_var = tk.StringVar()

    def update_dynamic_inputs(*args):
        for widget in dynamic_widgets_frame.winfo_children():
            widget.destroy()

        if mode_var.get() == "proportioned":
            ttk.Label(dynamic_widgets_frame, text="Initial State (e.g. {0:25,1:25,2:25,3:25})").pack()
            ttk.Entry(dynamic_widgets_frame, textvariable=input_state_var).pack()
            ttk.Label(dynamic_widgets_frame, text="Final State (e.g. {0:10,1:50,2:20,3:20})").pack()
            ttk.Entry(dynamic_widgets_frame, textvariable=final_state_var).pack()
        else:
            ttk.Label(dynamic_widgets_frame, text="Split Events (e.g. {10: [(0, 5)]})").pack()
            ttk.Entry(dynamic_widgets_frame, textvariable=split_events_var).pack()
            ttk.Label(dynamic_widgets_frame, text="Merge Events (e.g. {50: [(1, 0, 10)]})").pack()
            ttk.Entry(dynamic_widgets_frame, textvariable=merge_events_var).pack()

    mode_dropdown.bind("<<ComboboxSelected>>", update_dynamic_inputs)
    update_dynamic_inputs()

    left_canvas = ttk.Frame(left_frame)
    left_canvas.grid(row=9, column=0, columnspan=2, sticky='nsew', pady=10)

    def generate_and_visualize():
        global graphs_data
        try:
            if mode_var.get() == "proportioned":
                init_state = eval(input_state_var.get())
                final_state = eval(final_state_var.get())
                graphs_data, _, _ = generate_proportional_transition(
                    num_nodes=num_nodes_var.get(),
                    num_steps=num_steps_var.get(),
                    initial_state=init_state,
                    final_state=final_state,
                    intra_community_strength=intra_strength_var.get(),
                    inter_community_strength=inter_strength_var.get()
                )
                print("grahs generated with proportioned transition and initial state:", init_state, "final state:", final_state)
            else:
                split_ev = eval(split_events_var.get() or "{}")
                merge_ev = eval(merge_events_var.get() or "{}")
                graphs_data, _, _ = generate_dynamic_graphs(
                    num_nodes=num_nodes_var.get(),
                    num_steps=num_steps_var.get(),
                    initial_groups=1,
                    intra_community_strength=intra_strength_var.get(),
                    inter_community_strength=inter_strength_var.get(),
                    split_events=split_ev,
                    merge_events=merge_ev
                )
                print("grahs generated with split events:", split_ev, "merge events:", merge_ev)

            # --- Visualize reduced number of graphs ---
            for widget in left_canvas.winfo_children():
                widget.destroy()

            '''
            fig = plt.Figure(figsize=(9, 5))
            ax = fig.add_subplot(111)
            selected_keys = list(sorted(graphs_data.keys()))
            step = max(1, len(selected_keys) // 20)
            show_keys = selected_keys[::step]

            cols = 5
            rows = (len(show_keys) + cols - 1) // cols
            fig.set_size_inches(4 * cols, 3 * rows)
            fig.clf()

            for i, t in enumerate(show_keys):
                sub_ax = fig.add_subplot(rows, cols, i + 1)
                pos = nx.spring_layout(graphs_data[t])
                nx.draw(graphs_data[t], pos, with_labels=True, node_size=300, font_size=8, font_color='white', ax=sub_ax)
                sub_ax.set_title(f"t={t}", fontsize=8)

            canvas = FigureCanvasTkAgg(fig, master=left_canvas)
            canvas.draw()
            canvas.get_tk_widget().pack(fill='both', expand=True) 
            print("Graphs visualized")
            '''
            

            render_rug()
        except Exception as e:
            print("Error:", e)

    ttk.Button(left_frame, text="Generate & Visualize", command=generate_and_visualize).grid(row=8, column=0, columnspan=2, pady=10)

    # --- Right Panel (NetworkRug) ---
    color_options = ['id', 'id2', 'id3', 'custom', 'degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'eigenvector_centrality']
    colormap_options = ['turbo', 'gist_rainbow', 'ocean', 'rainbow', 'bwr', 'viridis', 'plasma', 'cividis']
    ordering_options = ['priority', 'dfs', 'bfs']

    color_var = tk.StringVar(value='closeness_centrality')
    colormap_var = tk.StringVar(value='turbo')
    order_var = tk.StringVar(value='priority')
    pixel_var = tk.IntVar(value=6)
    label_var = tk.BooleanVar()

    ttk.Label(right_frame, text="Color Encoding").pack()
    ttk.Combobox(right_frame, textvariable=color_var, values=color_options).pack()
    ttk.Label(right_frame, text="Colormap").pack()
    ttk.Combobox(right_frame, textvariable=colormap_var, values=colormap_options).pack()
    ttk.Label(right_frame, text="Ordering").pack()
    ttk.Combobox(right_frame, textvariable=order_var, values=ordering_options).pack()
    ttk.Checkbutton(right_frame, text="Show Labels", variable=label_var).pack()
    ttk.Label(right_frame, text="Pixel Size").pack()
    ttk.Scale(right_frame, from_=2, to=20, variable=pixel_var, orient='horizontal').pack()

    rug_canvas = ttk.Frame(right_frame)
    rug_canvas.pack(fill='both', expand=True)

    def render_rug():
        if not graphs_data:
            return

        fig = plt.Figure(figsize=(16, 9), dpi=100)
        ax = fig.add_subplot(111)

        ordering_fn = {
            'priority': get_priority_bfs_ordering,
            'dfs': get_DFS_ordering,
            'bfs': lambda g: get_BFS_ordering(g, sorting_key='weight')
        }[order_var.get()]

        ordering = ordering_fn(graphs_data)

        nrugs.draw_networkrug(
            graphs=graphs_data,
            color_encoding=color_var.get(),
            colormap=colormap_var.get(),
            labels=label_var.get(),
            pixel_size=pixel_var.get(),
            order=order_var.get(),
            ax=ax
        )

        for widget in rug_canvas.winfo_children():
            widget.destroy()

        canvas = FigureCanvasTkAgg(fig, master=rug_canvas)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)

    ttk.Button(right_frame, text="Render Rug", command=render_rug).pack(pady=10)
    root.mainloop()


In [5]:

start_gui()

grahs generated with split events: {10: [(0, 10)]} merge events: {}
num_artists 50
colormap created with turbo and 0 to 49
num_artists 50
colormap created with turbo and 0 to 49
